In [184]:
from contextlib import contextmanager
from threading  import Lock, Thread
from time import sleep
import operator as _

class SharedExclusive(object):

    def __init__(self):
        self.write_lock_obj = Lock()
        self.num_r = 0

    def r_acquire(self):
        print("\nRead_lock Called:")
        if self.write_lock_obj.locked():
            print("Wait for write_lock")
        else:
            print("Read lock")
            self.num_r += 1

    def r_release(self):
        if self.num_r > 0:
            print("Release read lock")
            self.num_r -= 1

    @contextmanager
    def r_locked(self):
        try:
            self.r_acquire()
            yield
        finally:
            self.r_release()

    def w_acquire(self):
        print("\nWrite_lock Called:")
        if self.num_r:
            print("Wait for read_lock")
        elif self.write_lock_obj.locked():
            print("Wait for write_lock")
            self.write_lock_obj.acquire()
        else:
            print('Write lock')
            self.write_lock_obj.acquire()

    def w_release(self):
        if self.write_lock_obj.locked():
            print('Release write lock')
            self.write_lock_obj.release()

    @contextmanager
    def w_locked(self):
        try:
            self.w_acquire()
            yield
        finally:
            self.w_release()
    
    def create_local_variable(self, block):
        variable = 0
        ldic = locals()
        try:
            exec(f"variable = self.{block['variable']}", ldic)
            # print(f"variable {block['variable']} already exists!")
        except AttributeError:
            exec(f"self.{block['variable']} = 0")
            exec(f"variable = self.{block['variable']}", ldic)
            # print(f"variable {block['variable']} created!")
        finally:
            variable = ldic['variable']
        return variable
    
    def create_transaction(self,transaction):
        for block in transaction:
            operation_name = block['operations'][0]
            if operation_name == 'read_item':
                with self.r_locked():
                    variable = self.create_local_variable(block)
                    print(variable)
                    
            elif operation_name == 'write_item':
                operation_task = block['operations'][1]
                with self.w_locked():
                    variable = self.create_local_variable(block)
                    print(variable)
                    operator, operand = operation_task.split()
                    mapper = {
                        '+' : _.add,
                        '-' : _.sub,
                        '*' : _.mul,
                        '/' : _.truediv,
                    }
                    func = mapper.get(operator, None)
                    if func:
                        variable = func(variable, float(operand))
                    sleep(1)
                    exec(f"self.{block['variable']} = variable")
                

In [191]:
lock = SharedExclusive()
transaction = [
    {
        'variable': "x",
        'operations': [
            'write_item', 
            '+ 10'
        ]
    },
    {
        'variable': "x",
        'operations': [
            'read_item', 
        ]
    }
]
# for simplicity we assume all transactions are same.
schedule = [transaction for _ in range(5)]

t1 = Thread(target=lock.create_transaction, args=(schedule[0],))
t2 = Thread(target=lock.create_transaction, args=(schedule[1],))
t3 = Thread(target=lock.create_transaction, args=(schedule[2],))
t4 = Thread(target=lock.create_transaction, args=(schedule[3],))
t5 = Thread(target=lock.create_transaction, args=(schedule[4],))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()

print(f"\nx = {lock.x}")
# print(f"y = {lock.y}")
print("Finished!")

[[{'variable': 'x', 'operations': ['write_item', '+ 10']}, {'variable': 'x', 'operations': ['read_item']}], [{'variable': 'x', 'operations': ['write_item', '+ 10']}, {'variable': 'x', 'operations': ['read_item']}], [{'variable': 'x', 'operations': ['write_item', '+ 10']}, {'variable': 'x', 'operations': ['read_item']}], [{'variable': 'x', 'operations': ['write_item', '+ 10']}, {'variable': 'x', 'operations': ['read_item']}], [{'variable': 'x', 'operations': ['write_item', '+ 10']}, {'variable': 'x', 'operations': ['read_item']}]]

Write_lock Called:
Write lock
0

Write_lock Called:
Wait for write_lock

Write_lock Called:
Wait for write_lock

Write_lock Called:
Wait for write_lock

Write_lock Called:
Wait for write_lock
Release write lock

Read_lock Called:
Read lock
10.0
Release read lock
10.0
Release write lock

Read_lock Called:
Read lock
20.0
Release read lock
20.0
Release write lock

Read_lock Called:
Read lock
30.0
Release read lock
30.0
Release write lock

Read_lock Called:
Read 